## Figure 1

In [1]:
import altair as alt
from vega_datasets import data
import pandas as pd
import numpy as np

from IPython.display import display, HTML

In [2]:
states = alt.topo_feature(data.us_10m.url, 'states')
cost_disability = pd.read_csv('cost_disability.csv')

In [6]:
state_names = cost_disability['LocationDesc'].copy()
state_ids = cost_disability['id'].copy()
state_ids = sorted(state_ids, key=lambda x: state_names[cost_disability['id'] == x].values[0])
state_names = sorted(state_names)
# selectState = alt.selection_single(
#     fields = ['LocationDesc'],
#     bind = alt.binding_select(
#         options = [None] + state_names,
#         labels = ['All'] + state_names,
#         name = 'Select State'
#     ),
#     resolve = 'union',
# )

In [20]:
viz_sel = alt.selection_single(
    fields=['id'], 
    empty='all',
    on = 'dblclick',
    bind = alt.binding_select(
        options = [None] + state_ids,
        labels = ['All'] + state_names,
        name = 'Select State'
    )
)

cost_rate_min = cost_disability['cost_rate'].min()
cost_rate_max = cost_disability['cost_rate'].max()


fig1_base = alt.Chart(states).mark_geoshape(
    stroke='white'
).project(
    type = 'albersUsa',
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(cost_disability, 'id', list(cost_disability.columns))
)

# op_sub_condition = alt.condition(selectState, alt.value(0.9), alt.value(0.2))
op_condition = alt.condition(viz_sel, alt.value(0.9), alt.value(0.2))

fig1 = fig1_base.encode(
    color = alt.Color(
        'disability_rate:Q', 
        bin=alt.BinParams(maxbins=5),
        scale=alt.Scale(scheme='yelloworangered'), 
        title='Disability Rate'
    ),
    opacity = op_condition,
    tooltip=['LocationDesc:N', 'disability_rate:Q', 'cost_rate:Q'],
).properties(
    title = alt.TitleParams(
        text = 'Disability status among adults 18 years of age or older in each state',
        fontSize = 16,
    ),
)


fig2_base = alt.Chart(cost_disability).encode(
    y = alt.Y('cost_rate:Q', axis=alt.Axis(title='Rate')),
    x = alt.X('LocationDesc:N', sort=alt.EncodingSortField(field='cost_rate', op='sum', order='descending')),
)    
    
fig2 = fig2_base.mark_bar().encode(
    opacity = op_condition,
    tooltip=['LocationDesc:N', 'disability_rate:Q', 'cost_rate:Q'],
).properties(
    title = alt.TitleParams(
        text = 'Could not see a doctor due to coset in the past 12 months among adults 18 years of age or older',
        fontSize = 16,
    ),
)

text = fig2.mark_text(baseline='middle', dy=-20).encode(
    text=alt.condition(viz_sel, alt.Text('cost_rate:Q', format='.1%'), alt.value(' ')),
    opacity = alt.condition(~viz_sel, alt.value(0), alt.value(1.0)),
)


(fig1 & (fig2 + text)).properties(
    title = {
        'text': 'Disability Rate and Expenditure in US in 2020', 
        'fontSize': 20, 
        'fontWeight': 'bold',
    },
).add_selection(viz_sel).resolve_scale(x="shared")

d:\Miniconda\envs\si649\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

## Figure 2

In [4]:
health_conditions = pd.read_csv('disease.csv')
health_conditions['disability'] = health_conditions['disability'].map({1: "with", 0: "without"})

In [7]:
base_bar = alt.Chart(health_conditions).encode(
    y = alt.Y('disability:N', title=None),
)


bars = base_bar.mark_bar().encode(
    x = alt.X('value:Q'),
    color = alt.Color('disability:N'),
)


confidence_interval = base_bar.mark_errorbar(extent='ci', color='darkred').encode(
    x=alt.X('ci1:Q', title='Rate'),
    x2='ci2:Q',
)


annotation = base_bar.mark_text(align='left', dx=10).encode(
    x = alt.X('value:Q'),
    text = alt.Text('value:Q', format='.1%')
)


(bars + confidence_interval + annotation).facet(
    row = alt.Row(
        'disease:N', 
        header=alt.Header(title='Health Conditions', labelOrient='top'),
        sort = alt.EncodingSortField(field='value', op='max', order='descending')),
).properties(
    title = "Health Conditions among Adults w/o disability in US in 2020",
)

alt.FacetChart(...)